In [0]:
#@title vox2music { display-mode: "form" }
#@test {"output": "ignore"}

print('Preparing...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta
!pip install soundfile
!pip install pydub

!git clone https://github.com/aurel-au-velin-olymp/v2m.git
!pip install colabutils

# !wget http://freepats.zenvoid.org/Piano/SalamanderGrandPiano/SalamanderGrandPianoV3+20161209_44khz16bit.tar.xz
# !tar -xf SalamanderGrandPianoV3+20161209_44khz16bit.tar.xz

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

from google.colab import files

%tensorflow_version 1.x
import magenta.music as mm
import magenta
import tensorflow

import mido
from mido import MidiFile
import librosa

import IPython.display as ipd
import matplotlib.pyplot as plt

import numpy as np
import os

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

print('🎉 Done!')


In [0]:
#@title Choose piece/composer { display-mode: "form" }
midi_name = "Beethoven Moonlight" #@param ["Beethoven Moonlight"]

midi_dict = {
    'Beethoven Moonlight' : 'v2m/mond_1_format0.mid',
    'Bach Prelude' : 'v2m/wtk1-prelude1.mid'
             }

mid = MidiFile(midi_dict[midi_name])

In [0]:
#@title Choose audio or record your own
wav_name =  "Kinski Laesterzungen" #@param ["Kinski Laesterzungen", "record your own"]

if wav_name != 'record your own':
  wav_dict = {
      "Kinski Laesterzungen" : 'v2m/klaus-kinski-liest-villon-die-lasterzungen.mp3'      
              }

  wav, sr = librosa.load(wav_dict[wav_name], sr=44100)

else:
  from colabutils import audio
  audio.record_and_save()
  wav, sr = librosa.load('audio.wav', sr=44100)

wav /= np.max(abs(wav))

In [0]:
min_velocity = 40 #@param {type:"integer"}
nrg = librosa.feature.rms(wav, center=True).repeat(512) * 100 + min_velocity


plt.plot(nrg)    
print('Preview of intensity curve:')
plt.show()


In [0]:
from mido import tick2second
from mido import Message, MidiFile, MidiTrack

print('Performing silently...')


frame = 0.5 #@param {type:"slider", min:-1, max:1, step:0.1}


velocity = 40
mods = [0]*10
vels = [velocity]*10
modulation = 0

current_time = 0


mid_new = MidiFile(ticks_per_beat=mid.ticks_per_beat)

for i in range(len(mid.tracks)):

  track = MidiTrack()
  mid_new.tracks.append(track)


  for msg in mid.tracks[i]:

    if msg.is_meta and 'tempo' in msg.type:
      #tempo = msg.tempo
      msg.tempo = 1000000

      
    #msg = msg
    
    #%store -r tempo
        
    
    
    tempo = (1 / (1 + modulation*0.01)) + 0.4
    #print('tempo', tempo)
    msg.time = int(msg.time * (tempo * 1.1))

        
    #time.sleep(msg.time)
    current_time += tick2second(msg.time, mid.ticks_per_beat, 1000000)
    current_sample = int(current_time*44100)
    #print('sample', current_sample)
    
    
    if current_time > frame:
      try:

        current_nrg = np.max(nrg[current_sample-int(frame*44100):current_sample+int(frame*44100)])
      except:
        current_nrg = 0

      
    else:
      current_nrg = np.mean(nrg[0:current_sample+int(frame*44100)])

    modulation = current_nrg
        
    #print('current_nrg', current_nrg)




    
    
    if not msg.is_meta and 'note' in msg.type:
        #print('note', msg.note)


        if msg.velocity > 0:
            
            #print('modulation', modulation)
            
            delta = False
            
            if delta == 'A':
                mod_delta = np.mean(np.diff(mods)[-1:])
                if mod_delta < 0:
                    velocity += 5
                else:
                    velocity = modulation
                    
            if delta == 'B':
                mod_delta = modulation - mods[-1]
                if mod_delta < 0:
                    velocity = int(vels[-1] * 0.25)
                else:
                    velocity = modulation
            
            if delta == False:
                velocity = modulation

            #print('velocity', velocity)
            velocity = int(np.clip(velocity, 20, 127))
            #print('velocity', velocity)

            msg.velocity = velocity


            

            
            #print(velocity)
            if msg.velocity < 10:
                msg.velocity = 10
                
            mods.append(modulation)
            vels.append(velocity)



    track.append(msg)

mid_new.save('new.mid')
print('Performance complete.')


In [0]:
#@title Choose Soundfont and render audio

print('Rendering audio. This may take up to 30 seconds...')

note_seq = mm.midi_file_to_note_sequence('new.mid')

sf2 = 'SalC5Light2.sf2' #@param['SalC5Light2.sf2', 'Salamander_C5-v3-HEDSounds.sf2']
music = mm.fluidsynth(note_seq, 44100, sf2_path='v2m/{}'.format(sf2))
mix = music.copy()
mix[:wav.shape[0]] += wav

mix /= np.max(abs(mix))

import soundfile as sf
import pydub

sf.write('output.wav', mix, 44100)
sound = pydub.AudioSegment.from_wav('output.wav')
sound.export("output.mp3", format="mp3")

ipd.Audio('output.mp3')